# Classificador Multi-layer Perceptron - Implantação

O [Multi-Layer Perceptron (MLP)](https://en.wikipedia.org/wiki/Multilayer_perceptron) é uma das classes mais tradicionais de redes neurais artificiais.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/28/MultiLayerPerceptron.png" alt="MLP" width="400"/>

Um MLP consiste em uma camada de entrada, uma ou mais camadas interemediárias, e uma camada de saída. Cada um dos nós tem uma [função de ativação](https://en.wikipedia.org/wiki/Activation_function), e são conhecidos como neurônios artificiais. O MLP utiliza também técnicas de Aprendizagem Supervisionada como [backpropagation](https://en.wikipedia.org/wiki/Backpropagation), para realizar o seu treinamento.

Este componente realiza predições usando um modelo Multi-layer Perceptron para classificação usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/mlp-classifier.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.label_encoder = artifacts["label_encoder"]
        self.method = artifacts["method"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate((self.columns_to_filter, self.new_columns))
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)[self.columns_to_filter]
            X = df.to_numpy()

        # adiciona a predição ao conjunto de dados
        y_prob = self.pipeline.predict_proba(X)
        y_pred = self.pipeline.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        
        df.loc[:, self.new_columns[:-1]] = y_prob
        df.loc[:, self.new_columns[-1]] = y_pred

        return df.to_numpy()

In [ ]:
dataset = "/tmp/data/MultiLayerPerceptron-2.png" #@param {type:"string"}